In [61]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType 
# you may add more import if you need to
from pyspark.context import SparkContext
from pyspark.sql import functions as F

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.appName("Assigment 2 Question 5").getOrCreate()
# YOUR CODE GOES BELOW
df = spark.read.parquet("./data/tmdb_5000_credits.parquet")
df =df.drop("crew")


In [62]:
df = df.withColumn("cast",F.split(df['cast'],'\\}, \\{"cast_id"'))
df = df.withColumn("cast",F.explode("cast"))

In [63]:
def extractName(data):
    print(data)
    movie_id = data.movie_id
    title = data.title
    cast = data.cast
    index = cast.find('"name"') + 9
    cast = cast[index:]
    cast = cast.strip()
    return (movie_id,title,cast)


df = df.withColumn("cast",F.split(df['cast'],'", "'))
df = df.withColumn("cast",F.explode("cast"))
print(df.count())
df = df.filter(F.col("cast").contains("name"))
print(df.count())
df.printSchema()
rdd = df.rdd.map(lambda x: extractName(x))
df = spark.createDataFrame(rdd).toDF("movie_id","title","actor")
df = df.sort("actor")


425071
106279
root
 |-- movie_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: string (nullable = true)



In [64]:
def countMovies(data):
    actor = data.actor
    title = data['collect_list(title)']
    movie_id = data['collect_list(movie_id)']
    c = len(movie_id)
    return (actor,title,movie_id,c)
df2 = df.groupBy('movie_id','title').agg(F.collect_list('actor')).sort('movie_id')
df2.show()
#df = df.groupBy('actor').agg(F.collect_list('title'),F.collect_list('movie_id'))


+--------+--------------------+--------------------+
|movie_id|               title| collect_list(actor)|
+--------+--------------------+--------------------+
|       5|          Four Rooms|[Alicia Witt, Ama...|
|      11|           Star Wars|[Al Lampert, Alan...|
|      12|        Finding Nemo|[Albert Brooks, A...|
|      13|        Forrest Gump|[Aaron Michael La...|
|      14|     American Beauty|[Elaine Corral Ke...|
|      16|  Dancer in the Dark|[Al Agami, Anna N...|
|      18|   The Fifth Element|[Eddie Ellwood, F...|
|      19|          Metropolis|[Alfred Abel, Art...|
|      20|  My Life Without Me|[Alfred Molina, A...|
|      22|Pirates of the Ca...|[Angus Barnett, A...|
|      24|   Kill Bill: Vol. 1|[Ai Maeda, Akaji ...|
|      25|             Jarhead|[Brian Geraghty, ...|
|      28|      Apocalypse Now|[Albert Hall, Aur...|
|      33|          Unforgiven|[Anna Levine, Ant...|
|      35|  The Simpsons Movie|[Albert Brooks, B...|
|      38|Eternal Sunshine ...|[Amir Ali Said,

In [65]:
df2 = df2.select('movie_id','title',F.explode('collect_list(actor)').alias('actor'))
df2.show()

+--------+----------+-----------------+
|movie_id|     title|            actor|
+--------+----------+-----------------+
|       5|Four Rooms|      Alicia Witt|
|       5|Four Rooms|Amanda de Cadenet|
|       5|Four Rooms| Antonio Banderas|
|       5|Four Rooms|     Bruce Willis|
|       5|Four Rooms|   Danny Verduzco|
|       5|Four Rooms|     David Proval|
|       5|Four Rooms|        Ione Skye|
|       5|Four Rooms|   Jennifer Beals|
|       5|Four Rooms|    Kathy Griffin|
|       5|Four Rooms|   Kimberly Blair|
|       5|Four Rooms|   Lana McKissack|
|       5|Four Rooms|  Lawrence Bender|
|       5|Four Rooms|      Lili Taylor|
|       5|Four Rooms|          Madonna|
|       5|Four Rooms|    Marc Lawrence|
|       5|Four Rooms|     Marisa Tomei|
|       5|Four Rooms|   Patricia Vonne|
|       5|Four Rooms|    Paul Calderon|
|       5|Four Rooms|Quentin Tarantino|
|       5|Four Rooms|      Salma Hayek|
+--------+----------+-----------------+
only showing top 20 rows



In [66]:
df2.count()

106279

In [67]:
df2 = df2.distinct()
df2.count()

106095

In [ ]:
df3 = df2.alias('df3')
df4 = df2.alias('df4')
df3 = df3.withColumnRenamed("actor","actor1")
df4 = df4.withColumnRenamed("actor","actor2")

dfForJoining = df3.join(df4,[F.col("df3.movie_id")==F.col("df4.movie_id"),F.col("actor1")<F.col("actor2")])

In [ ]:
df =dfForJoining.groupBy("actor1","actor2").agg(F.count("df3.movie_id"))
df.show()

In [ ]:
df = df.filter(df["`count(df3.movie_id)`"]>1)
df.show()

In [ ]:
dfForJoining = dfForJoining.select("df3.movie_id","df3.title","actor1","actor2")
dfForJoining = dfForJoining.withColumnRenamed("actor1","joinActor1")
dfForJoining = dfForJoining.withColumnRenamed("actor2","joinActor2")

df = df.join(dfForJoining,[F.col("actor1")==F.col("joinActor1"),F.col("actor2")==F.col("joinActor2")])
df.show()

In [ ]:
df = df.select('movie_id','title','actor1','actor2')
df.show()
print(df.count())

In [68]:
#df.toPandas().to_csv("output/modifiedData5.csv")
df.toPandas().to_parquet("output/modifiedData5.parquet")

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
df.write.parquet("output/modifiedData5.parquet")